In [ ]:
import os
import glob
import pandas as pd 
import random
import Levenshtein
import numpy as np
from scipy.io import savemat

# กำหนด paths
path_benign = "C:\\Users\\BMEI CMU\\Documents\\DikeASCII_DATASET\\Benign"
path_malware = "C:\\Users\\BMEI CMU\\Documents\\DikeASCII_DATASET\\Malware"

# ฟังก์ชันในการอ่านไฟล์ .txt และแทนที่ช่องว่างด้วย '0' และรวมข้อมูลเป็นแถวเดียว พร้อมเพิ่มคอลัมน์เลข
def read_and_replace_spaces_as_rows(folder_path, label):
    # ค้นหาไฟล์ .txt ทั้งหมดในโฟลเดอร์และเลือกเฉพาะไฟล์ที่ต้องการ
    file_paths = glob.glob(os.path.join(folder_path, "*.txt"))[:100]
    modified_data_rows = []
    
    for file_path in file_paths:
        # อ่านเนื้อหาจากไฟล์แต่ละบรรทัดและรวมบรรทัดทั้งหมดไว้ในแถวเดียว
        with open(file_path, 'r', encoding='utf-8') as file:
            data = file.read().replace('\n', '')  # รวมบรรทัดทั้งหมดเป็นแถวเดียว
        
        # แทนที่ช่องว่างด้วย '0' ในแถวเดียว
        modified_data = ''.join(['0' if char == ' ' else char for char in data])
        
        # เพิ่มข้อมูลเป็น dictionary พร้อม label เพื่อแสดงเป็นคอลัมน์
        modified_data_rows.append({"data": modified_data, "label": label})

    return modified_data_rows

# อ่านและแทนที่ช่องว่างในไฟล์จากโฟลเดอร์ benign และ malware พร้อม label
benign_data = read_and_replace_spaces_as_rows(path_benign, 0)
malware_data = read_and_replace_spaces_as_rows(path_malware, 1)

# แปลงข้อมูลเป็น DataFrame แยกกัน
benign_df = pd.DataFrame(benign_data)
malware_df = pd.DataFrame(malware_data)

In [2]:
# กำหนดจำนวนแถวที่จะสุ่ม
num_samples_benign = 10
num_samples_malware = 10

# สุ่มเลือกแถวจาก benign_data พร้อมเก็บ index ของแถวที่เลือก
benign_sample_indices = random.sample(range(len(benign_df)), num_samples_benign)
benign_sampled_data = benign_df.iloc[benign_sample_indices].reset_index(drop=True)

# สุ่มเลือกแถวจาก malware_data พร้อมเก็บ index ของแถวที่เลือก
malware_sample_indices = random.sample(range(len(malware_df)), num_samples_malware)
malware_sampled_data = malware_df.iloc[malware_sample_indices].reset_index(drop=True)


In [3]:
# สร้าง DataFrame สำหรับ benign และ malware ที่เหลือเป็น TEST โดยใช้ดัชนีที่ไม่ได้ถูกสุ่มมา
benign_test_df = benign_df.drop(benign_sample_indices).reset_index(drop=True)
malware_test_df = malware_df.drop(malware_sample_indices).reset_index(drop=True)

# รวม TRAIN และ TEST สำหรับ benign และ malware
train_df = pd.concat([benign_sampled_data, malware_sampled_data], ignore_index=True)
test_df = pd.concat([benign_test_df, malware_test_df], ignore_index=True)

In [ ]:
# สมมติว่า train_df มีอยู่แล้ว
# สร้างตารางเปล่าสำหรับเก็บผลลัพธ์ Edit Distance
n = len(train_df)
edit_distance_matrix = pd.DataFrame(index=range(n), columns=range(n))

# คำนวณ Edit Distance แบบ nxn
for i in range(n):
    for j in range(n):
        if i != j:
            edit_distance_matrix.iloc[i, j] = Levenshtein.distance(train_df['data'][i], train_df['data'][j])
        else:
            edit_distance_matrix.iloc[i, j] = 0  # ค่าเดียวกัน ให้เป็น 0

# แปลง DataFrame เป็น numpy array
edit_distance_array = edit_distance_matrix.to_numpy()

# กำหนด path สำหรับบันทึก
output_path = r"C:\Users\BMEI CMU\Desktop\RESULT\edit_distance_matrix.mat"

# บันทึกเป็นไฟล์ .mat
savemat(output_path, {'edit_distance_matrix': edit_distance_array})

print(f"Edit Distance Matrix saved as .mat at {output_path}")


KeyboardInterrupt: 